### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
from pathlib import Path

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,year,school_name,reading_score,maths_score,School ID,type,size,budget
0,0,Paul Bradley,M,9,Huang High School,96,94,0,Government,2917,1910635
1,1,Victor Smith,M,12,Huang High School,90,43,0,Government,2917,1910635
2,2,Kevin Rodriguez,M,12,Huang High School,41,76,0,Government,2917,1910635
3,3,Richard Scott,M,12,Huang High School,89,86,0,Government,2917,1910635
4,4,Bonnie Ray,F,9,Huang High School,87,69,0,Government,2917,1910635


## Local Government Area Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average maths score 

* Calculate the average reading score

* Calculate the percentage of students with a passing maths score (50 or greater)

* Calculate the percentage of students with a passing reading score (50 or greater)

* Calculate the percentage of students who passed maths **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
# Creating lists with data that doesn't need extensive calculations
total_schools = len(school_data["School ID"])
total_students = len(school_data_complete["Student ID"])
total_budget = school_data["budget"].sum()
average_math_scores = school_data_complete["readings_score"].mean()
average_reading_scores = school_data_complete["reading_score"].mean()

# The two lines below first finds the total number below 50 and then finds the percentage of the total amount
passing_math = len(school_data_complete[school_data_complete["maths_score"] > 50])/len(school_data_complete["maths_score"])*100
passing_reading = len(school_data_complete[school_data_complete["reading_score"] > 50])/len(school_data_complete["reading_score"])*100

# Creating the LGA_summary df
LGA_summary = pd.DataFrame({"Total Schools" : [total_schools], "Total Students" : [total_students], "Total Budget" : [total_budget],
                            "Average Math Scores" : [average_math_scores], "Average Reading Scores" : [average_reading_scores],
                            "% Passing Maths" : [passing_math], "% Passing Reading" : [passing_reading]})

# This block of code formats each number 
LGA_summary["Total Students"] = LGA_summary["Total Students"].map("{:,.0f}".format)
LGA_summary["Total Budget"] = LGA_summary["Total Budget"].map("{:,.0f}".format)
LGA_summary["Average Math Scores"] = LGA_summary["Average Math Scores"].map("{:,.2f}".format)
LGA_summary["Average Reading Scores"] = LGA_summary["Average Reading Scores"].map("{:,.2f}".format)
LGA_summary["% Passing Maths"] = LGA_summary["% Passing Maths"].map("{:,.2f} %".format)
LGA_summary["% Passing Reading"] = LGA_summary["% Passing Reading"].map("{:,.2f} %".format)

LGA_summary

,Total Schools,Total Students,Total Budget,Average Math Scores,Average Reading Scores,% Passing Maths,% Passing Reading
0,15,"39,170","24,649,428",70.34,69.98,84.43 %,82.64 %


## School Summary

* Create an overview table that summarises key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Maths Score
  * Average Reading Score
  * % Passing Maths
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed maths **and** reading.)
  
* Create a dataframe to hold the above results

In [78]:
# Creating an empty dataframe to put all the values in
school_summary_df = pd.DataFrame({"School Name" : [], "School Type" : [], "Total Students" : [], "Total School Budget" : [],
                                  "Per Student Budget" : [], "Average Maths Score" : [],
                                  "Average Reading Score" : [], "% Passing Maths" : [], "% Passing Reading" : [],
                                  "% Overall Passing" : []})

# This section allocates all of the columns in school_summary_df that don't need extensive calculations
school_summary_df["School Name"] = school_data["school_name"]
school_summary_df["School Type"] = school_data["type"]
school_summary_df["Total Students"] = school_data["size"]
school_summary_df["Total School Budget"] = school_data["budget"]
school_summary_df["Per Student Budget"] = school_summary_df["Total School Budget"]/school_summary_df["Total Students"]
school_summary_df["Average Maths Score"] = school_data_complete["maths_score"]
school_summary_df["Average Reading Score"] = school_data_complete["reading_score"]

# Created a new dataframe which sorts by school_name to find values above 50
school_sorted_name = school_data_complete.sort_values(by="school_name")

# The next four lines apply a lambda function to the school_sorted_name dataframe which counts each row above 50, finds the average of that calculation,
# and then multiplies by 100 to get the percentage
school_passing_maths = school_sorted_name.groupby("school_name")["maths_score"].apply(lambda x: (x > 50).mean() * 100).reset_index()
school_summary_df["% Passing Maths"] = school_passing_maths["maths_score"]

school_passing_reading = school_sorted_name.groupby("school_name")["reading_score"].apply(lambda x: (x > 50).mean() * 100).reset_index()
school_summary_df["% Passing Reading"] = school_passing_reading["reading_score"]

# This section creates a new column in the school_sorted_name dataframe which then performs the same function as the above section using .gt to get vales greater than 50
school_overall_passing = school_sorted_name.assign(overall_score = school_sorted_name[["maths_score", "reading_score"]].gt(50).mean(axis=1) * 100).groupby("school_name")["overall_score"].mean().reset_index()
school_summary_df["% Overall Passing"] = school_overall_passing["overall_score"]

# This section formats numbers over 100 and percentages for readability
school_summary_df["Total Students"] = school_summary_df["Total Students"].map("{:,.0f}".format)
school_summary_df["Total School Budget"] = school_summary_df["Total School Budget"].map("{:,.0f}".format)
school_summary_df["Per Student Budget"] = school_summary_df["Per Student Budget"].map("{:,.0f}".format)
school_summary_df["% Passing Maths"] = school_summary_df["% Passing Maths"].map("{:,.2f} %".format)
school_summary_df["% Passing Reading"] = school_summary_df["% Passing Reading"].map("{:,.2f} %".format)
school_summary_df["% Overall Passing"] = school_summary_df["% Overall Passing"].map("{:,.2f} %".format)


school_summary_df = school_summary_df.set_index("School Name")
school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Huang High School,Government,"2,917","1,910,635",655,94,96,89.95 %,85.41 %,87.68 %
Figueroa High School,Government,"2,949","1,884,411",639,43,90,88.81 %,86.71 %,87.76 %
Shelton High School,Independent,"1,761","1,056,600",600,76,41,80.26 %,80.74 %,80.50 %
Hernandez High School,Government,"4,635","3,022,020",652,86,89,81.09 %,80.72 %,80.91 %
Griffin High School,Independent,"1,468","917,500",625,69,87,89.37 %,86.99 %,88.18 %
Wilson High School,Independent,"2,283","1,319,574",578,93,88,79.37 %,79.98 %,79.68 %
Cabrera High School,Independent,"1,858","1,081,356",582,60,73,88.29 %,86.65 %,87.47 %
Bailey High School,Government,"4,976","3,124,928",628,64,69,80.25 %,79.81 %,80.03 %
Holden High School,Independent,427,"248,087",581,66,94,80.30 %,80.53 %,80.41 %


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [37]:
top_schools_overall = school_summary_df.sort_values(by="% Overall Passing", ascending=False)
top_schools_overall.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Griffin High School,Independent,"1,468","917,500",625,69,87,89.37 %,86.99 %,88.18 %
Figueroa High School,Government,"2,949","1,884,411",639,43,90,88.81 %,86.71 %,87.76 %
Pena High School,Independent,962,"585,858",609,77,64,90.23 %,85.24 %,87.73 %
Huang High School,Government,"2,917","1,910,635",655,94,96,89.95 %,85.41 %,87.68 %
Thomas High School,Independent,"1,635","1,043,130",638,79,80,90.50 %,84.50 %,87.50 %


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [38]:
bottom_schools_overall = school_summary_df.sort_values(by="% Overall Passing", ascending=True)
bottom_schools_overall.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Wilson High School,Independent,"2,283","1,319,574",578,93,88,79.37 %,79.98 %,79.68 %
Bailey High School,Government,"4,976","3,124,928",628,64,69,80.25 %,79.81 %,80.03 %
Ford High School,Government,"2,739","1,763,916",644,88,73,80.73 %,79.89 %,80.31 %
Holden High School,Independent,427,"248,087",581,66,94,80.30 %,80.53 %,80.41 %
Shelton High School,Independent,"1,761","1,056,600",600,76,41,80.26 %,80.74 %,80.50 %


## Maths Scores by Year

* Create a table that lists the average maths score for students of each year level (9, 10, 11, 12) at each school.

  * Create a pandas series for each year. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [93]:
# Creating dataframes with booleans for each year and then dropping all data not associated with the year
year_9_bool = school_data_complete.assign(year_9 = school_data_complete["year"].eq(9))
year_9_bool = year_9_bool.drop(year_9_bool[year_9_bool["year_9"] == False].index)

year_10_bool = school_data_complete.assign(year_10 = school_data_complete["year"].eq(10))
year_10_bool = year_10_bool.drop(year_10_bool[year_10_bool["year_10"] == False].index)

year_11_bool = school_data_complete.assign(year_11 = school_data_complete["year"].eq(11))
year_11_bool = year_11_bool.drop(year_11_bool[year_11_bool["year_11"] == False].index)

year_12_bool = school_data_complete.assign(year_12 = school_data_complete["year"].eq(12))
year_12_bool = year_12_bool.drop(year_12_bool[year_12_bool["year_12"] == False].index)

# Creating a dataframe to store all the results
math_scores_df = pd.DataFrame({"School Name" : [], "Year 9" : [], "Year 10" : [], "Year 11" : [], "Year 12" : []})

# Adding data to math_scores_df
math_scores_df["School Name"] = school_data["school_name"]
math_scores_df["Year 9"] = year_9_bool.groupby("school_name")["maths_score"].mean().reset_index()["maths_score"]
math_scores_df["Year 10"] = year_10_bool.groupby("school_name")["maths_score"].mean().reset_index()["maths_score"]
math_scores_df["Year 11"] = year_11_bool.groupby("school_name")["maths_score"].mean().reset_index()["maths_score"]
math_scores_df["Year 12"] = year_12_bool.groupby("school_name")["maths_score"].mean().reset_index()["maths_score"]

# Formatting
math_scores_df["Year 9"] = math_scores_df["Year 9"].map("{:,.2f}".format)
math_scores_df["Year 10"] = math_scores_df["Year 10"].map("{:,.2f}".format)
math_scores_df["Year 11"] = math_scores_df["Year 11"].map("{:,.2f}".format)
math_scores_df["Year 12"] = math_scores_df["Year 12"].map("{:,.2f}".format)

math_scores_df = math_scores_df.set_index("School Name")
math_scores_df

,Year 9,Year 10,Year 11,Year 12
School Name,,,,
Huang High School,72.49,71.90,72.37,72.68
Figueroa High School,72.32,72.44,71.01,70.60
Shelton High School,68.48,68.33,68.81,69.33
Hernandez High School,69.02,69.39,69.25,68.62
Griffin High School,72.79,71.09,71.69,71.47
Wilson High School,68.59,68.87,69.15,68.99
Cabrera High School,70.54,75.11,71.64,73.41
Bailey High School,69.08,68.53,69.43,68.64
Holden High School,69.47,67.99,68.64,69.29


## Reading Score by Year

* Perform the same operations as above for reading scores

In [94]:
# Creating a dataframe to store all the results
reading_scores_df = pd.DataFrame({"School Name" : [], "Year 9" : [], "Year 10" : [], "Year 11" : [], "Year 12" : []})

# Adding data to reading_scores_df
reading_scores_df["School Name"] = school_data["school_name"]
reading_scores_df["Year 9"] = year_9_bool.groupby("school_name")["reading_score"].mean().reset_index()["reading_score"]
reading_scores_df["Year 10"] = year_10_bool.groupby("school_name")["reading_score"].mean().reset_index()["reading_score"]
reading_scores_df["Year 11"] = year_11_bool.groupby("school_name")["reading_score"].mean().reset_index()["reading_score"]
reading_scores_df["Year 12"] = year_12_bool.groupby("school_name")["reading_score"].mean().reset_index()["reading_score"]

# Formatting
reading_scores_df["Year 9"] = reading_scores_df["Year 9"].map("{:,.2f}".format)
reading_scores_df["Year 10"] = reading_scores_df["Year 10"].map("{:,.2f}".format)
reading_scores_df["Year 11"] = reading_scores_df["Year 11"].map("{:,.2f}".format)
reading_scores_df["Year 12"] = reading_scores_df["Year 12"].map("{:,.2f}".format)

reading_scores_df = reading_scores_df.set_index("School Name")
reading_scores_df

,Year 9,Year 10,Year 11,Year 12
School Name,,,,
Huang High School,70.90,70.85,70.32,72.20
Figueroa High School,71.17,71.33,71.20,71.86
Shelton High School,70.26,67.68,69.15,69.08
Hernandez High School,69.62,68.99,70.74,68.85
Griffin High School,72.03,70.75,72.39,69.43
Wilson High School,68.48,70.62,68.42,69.24
Cabrera High School,71.60,71.10,73.31,70.48
Bailey High School,68.67,69.52,68.74,68.67
Holden High School,68.72,69.30,69.97,67.99


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Maths Score
  * Average Reading Score
  * % Passing Maths
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,71.36,70.72,88.84,86.39,76.72
$585-630,72.07,71.03,91.52,87.29,79.88
$630-645,69.85,69.84,84.69,83.76,71.00
$645-680,68.88,69.05,81.57,81.77,66.76


## Scores by School Size

* Perform the same operations as above, based on school size.

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),72.335748,71.636864,90.806867,87.557513,79.066348
Medium (1000-2000),71.421650,70.720164,89.846560,86.714149,78.039785
Large (2000-5000),69.751809,69.576052,84.252804,83.301185,70.293507


## Scores by School Type

* Perform the same operations as above, based on school type

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Type,,,,,
Government,69.834806,69.675929,84.462375,83.587562,70.698993
Independent,71.368822,70.718933,89.204043,86.247789,76.973340
